This notebook calculates and visualizes the year of peak water predicted by each glacier model and each individual GCM. The notebook is also used to create some multi-basin runoff figures.

Last edited: April 10, 2024 | FFW

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr
from cycler import cycler
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from matplotlib.colors import Normalize, ListedColormap
import statistics
from scipy.stats import linregress

#from datetime import datetime

In [ ]:
scenarios = ['ssp126','ssp245','ssp370','ssp585']

modelnames = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 
                  'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

regions = ['RGI 01', 'RGI 02', 'RGI 06', 'RGI 08', 'RGI 10', 'RGI 11', 'RGI 12', 
           'RGI 13', 'RGI 15', 'RGI 16', 'RGI 17', 'RGI 18']

gmodels = ['GloGEM', 'OGGM', 'PyGEM']

fpath0 = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/'

In [ ]:
basins = {'ALSEK':'4401', 'COLUMBIA':'4406','COLVILLE':'4110','COPPER':'4408', 'FRASER':'4410','KUSKOKWIM':'4414',
          'MACKENZIE':'4123','NASS':'4416', 'NUSHAGAK':'4418','NELSON':'4125', 'SKAGIT':'4426', 'SKEENA':'4427',
          'STIKINE':'4428','SUSITNA':'4430','TAKU':'4431',  'YUKON':'4435',
          
           
         'DANUBE':'6202', 'DRAMSELVA':'6209','GLOMAA':'6213','JOKULSA A FJOLLUM':'6101','KALIXALVEN':'6219',
          'LAGARFLJOT':'6104', 'LULEALVEN':'6227','OLFUSA':'6237','PO':'6241','RHINE':'6242', 
          'RHONE':'6243','SVARTA':'6110', 'THJORSA':'6254','TORNEALVEN':'6255',
         

          'ARAL SEA':'2902', 'BRAHMAPUTRA':'2302', 'CHUY':'2905', 'HAR US NUUR':'2909','GANGES':'2306','INDIGIRKA':'2103',
          'INDUS':'2309', 'IRRAWADDY':'2310','KAMCHATKA':'2413', 'KUBAN':'6223','LAKE BALKHASH':'2910','MEKONG':'2421',
          'OB':'2108','SALWEEN':'2319','TALAS':'2913','TARIM HE':'2914', 'UVS NUUR':'2918','YANGTZE' : '2433',
          'YELLOW RIVER':'2434','YSYK-KOL':'2919',  

          
          'AISEN':'3401','AMAZON':'3203','AZOPARDO':'3403','BAKER':'3404','BIOBIO':'3405','CHICO':'3209', 
          'CISNES':'3408','COLORADO':'3212','COPIAPO':'3409','HUASCO':'3412', 'MAGDALENA':'3227', 'MAJES':'3416',
          'NEGRO':'3232','OCONA':'3418','PALENA':'3419','PASCUA':'3420','PUELO':'3422','RAPEL':'3423',
          'SANTA':'3425','SANTA CRUZ':'3244','SERRANO':'3426','TITICACA':'3912','VALDIVIA':'3428','YELCHO':'3429', 
            
          'CLUTHA':'5406'}

In [ ]:
# Create new index using pandas date_range function
start_date = datetime.date(2000, 1, 1)
end_date = datetime.date(2100, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

In [ ]:
#Importing runoff data
all_rf_data = {}
all_rf_data_annual = {}

for r, region in enumerate(regions):
    fpath = fpath0 + 'Runoff/' + region + '/'
    
    for s, SSP in enumerate(scenarios):
        if SSP not in all_rf_data:
            all_rf_data[SSP] = {}
            all_rf_data_annual[SSP] = {}
        
        for b, basin in enumerate(basins):
            if basin not in all_rf_data[SSP]:
                all_rf_data[SSP][basin] = {}
                all_rf_data_annual[SSP][basin] = {}
            
            for m, GCM in enumerate(modelnames):
                fname = f"runoff_{GCM}_{SSP}_{basin}.csv"
                file_path = os.path.join(fpath, fname)  # Construct the full file path

                if os.path.exists(file_path):
                    try:
                        temp_df = pd.read_csv(file_path, index_col=0)
                        temp_df.index = new_indices
                        temp_df.index = pd.to_datetime(new_indices)
                        
                        all_rf_data[SSP][basin][GCM] = temp_df
                        all_rf_data_annual[SSP][basin][GCM] = temp_df.resample('A').sum()
                    except FileNotFoundError:
                        print(fname)
                        continue

In [ ]:
#Determining the year of peak water with savgol filter
# from scipy.signal import savgol_filter

# window_length = 15
# polyorder = 5

# peak_water_year = {}
# peak_water_dt = {}
# smoothed_runoff = {}
# for s, SSP in enumerate(scenarios):
#     peak_water_year[SSP] = {}
#     peak_water_dt[SSP] = {}
#     smoothed_runoff[SSP] = {}
#     for b, basin in enumerate(basins):
#         peak_water_year[SSP][basin] = {}
#         peak_water_dt[SSP][basin] = {}
#         smoothed_runoff[SSP][basin] = {}
#         for m, GCM in enumerate(modelnames):
#             df = all_rf_data_annual[SSP][basin][GCM]
#             smoothed_runoff[SSP][basin][GCM] = df[0:100].apply(lambda col: savgol_filter(col, window_length, polyorder))
#             #smoothed_runoff[SSP][basin][GCM]['OGGM'] = df[0:100].apply(lambda col: savgol_filter(col, window_length, polyorder))
#             peak_water_dt[SSP][basin][GCM] = smoothed_runoff[SSP][basin][GCM].idxmax()
#             peak_water_year[SSP][basin][GCM] = smoothed_runoff[SSP][basin][GCM].idxmax()
#             for gmodel in gmodels:
#                  peak_water_year[SSP][basin][GCM][gmodel]= peak_water_year[SSP][basin][GCM][gmodel].to_pydatetime().year

In [ ]:
#Creating time values and color schemes
yrs = np.arange(2000, 2100)
yrs_dt = pd.to_datetime([str(y) for y in yrs])

colorschemes = {}

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)
colorschemes['GloGEM'] = glo_cycler

colors_OG =  plt.colormaps['Blues']
line_colors_OG = colors_OG(np.linspace(0.2, 0.6,num = 12))
OG_cycler = cycler(color = line_colors_OG)
colorschemes['OGGM'] = OG_cycler

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)
colorschemes['PyGEM'] = py_cycler

colors = ['darkgreen', 'royalblue', 'purple']
fill_colors = ['green', 'dodgerblue', 'purple']

In [ ]:
# num_rows = 1
# num_cols = 4

# figy, axy = plt.subplots(num_rows, num_cols, figsize=(10, 4))
# row_max_values = []
# r = -1
# c = 0

# basins_slice = [ 'SANTA CRUZ', 'RHINE', 'ARAL SEA', 'YUKON']

# for b, basin in enumerate(basins_slice):
#     if c % 3 == 0:
#         r += 1  
#     c = (c + 1) % num_cols
#     for g, gmodel in enumerate(gmodels):
#         axy[c].plot(yrs_dt, smoothed_runoff['ssp585'][basin]['BCC-CSM2-MR'][gmodel], label = gmodel, color = colors[g])
#         axy[c].plot(peak_water_dt['ssp585'][basin]['BCC-CSM2-MR'][gmodel], smoothed_runoff['ssp585'][basin]['BCC-CSM2-MR'][gmodel].loc[peak_water_dt['ssp585'][basin]['BCC-CSM2-MR'][gmodel]], marker = 'o', color = fill_colors[g])

#         axy[c].set_xlabel('Year')
#         axy[c].set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2000, 2025, 2050, 2075, 2100])

#         if c == 0:                                                                    #Setting basin labels
#             axy[c].set_ylabel('Smoothed runoff' + r' $[km^3]$')
        
#     #Adding in text of which basin
#     row_max = max(axy[c].get_ylim())
#     row_max_values.append(row_max)
    
#     y_coord = row_max_values[b] * 1.028
#     axy[c].text(pd.to_datetime('2038-01-01'), y_coord, basin)
    
                      
# green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
# blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
# purple_patch = mpatches.Patch(color='purple', label='PyGEM') 

# axy[0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.15, (0.0167*len(basins))), ncol=3)

# plt.suptitle('Smoothed Runoff Projections and Calculated Peak Water Times for all Basins–SavGol Filter', x=0.52, y=1.12)

In [ ]:
#Examing our smoothed datasets and the assigned peak water times
# num_rows = 25
# num_cols = 3

# for s, SSP in enumerate(scenarios):
#     for m, GCM in enumerate(modelnames):
        
#         figy, axy = plt.subplots(num_rows, num_cols, figsize=(10, 0.8*len(basins)))
#         row_max_values = []
#         r = -1
#         c = 0
        
#         for b, basin in enumerate(basins):
#             if c % 3 == 0:
#                 r += 1  
#             c = (c + 1) % num_cols
#             for g, gmodel in enumerate(gmodels):
#                 axy[r,c].plot(yrs_dt, smoothed_runoff[SSP][basin][GCM][gmodel], label = gmodel, color = colors[g])
#                 axy[r,c].plot(peak_water_dt[SSP][basin][GCM][gmodel], smoothed_runoff[SSP][basin][GCM][gmodel].loc[peak_water_dt[SSP][basin][GCM][gmodel]], marker = 'o', color = fill_colors[g])
                
#             #Adding in text of which basin
#             row_max = max(axy[r, c].get_ylim())
#             row_max_values.append(row_max)
            
#             y_coord = row_max_values[b] * 0.92
#             axy[r,c].text(pd.to_datetime('2038-01-01'), y_coord, basin)
                              
#         green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
#         blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
#         purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
        
#         axy[0,0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(2.46, (0.0175*len(basins))), ncol=3)
        
#         plt.suptitle('Smoothed Runoff Projections and Calculated Peak Water Times for all Basins', x=0.52, y=0.892)
        
#         # Save the plot to your desktop
#         desktop_path = os.path.expanduser("~/Desktop/Lizz Research/Figures/Peak Water/")  # Get the path to your desktop
#         file_name = f"PeakWater{GCM}_{SSP}.png"  # Name of the file to save
        
        
#         # Combine the desktop path and file name to create the full file path
#         save_path = os.path.join(desktop_path, file_name)
        
#         # Save the plot as an image
#         plt.savefig(save_path, dpi=300, bbox_inches='tight')

#         plt.close(figy)

In [ ]:
#Determining the year of peak water with rolling mean

peak_water_year = {}
peak_water_dt = {}
smoothed_runoff = {}
for s, SSP in enumerate(scenarios):
    peak_water_year[SSP] = {}
    peak_water_dt[SSP] = {}
    smoothed_runoff[SSP] = {}
    for b, basin in enumerate(basins):
        peak_water_year[SSP][basin] = {}
        peak_water_dt[SSP][basin] = {}
        smoothed_runoff[SSP][basin] = {}
        for m, GCM in enumerate(modelnames):
            df = all_rf_data_annual[SSP][basin][GCM]
            smoothed_runoff[SSP][basin][GCM]= df.rolling(20, center = True).mean()
            peak_water_dt[SSP][basin][GCM] = smoothed_runoff[SSP][basin][GCM].idxmax()
            peak_water_year[SSP][basin][GCM] = smoothed_runoff[SSP][basin][GCM].idxmax()
            for gmodel in gmodels:
                 peak_water_year[SSP][basin][GCM][gmodel]= peak_water_year[SSP][basin][GCM][gmodel].to_pydatetime().year

In [ ]:
num_rows = 1
num_cols = 4

figy, axy = plt.subplots(num_rows, num_cols, figsize=(10, 4))
row_max_values = []
r = -1
c = 0

basins_slice = [ 'BAKER', 'RAPEL', 'CHICO', 'YUKON']

for b, basin in enumerate(basins_slice):
    if c % 3 == 0:
        r += 1  
    c = (c + 1) % num_cols
    for g, gmodel in enumerate(gmodels):
        axy[c].plot(yrs_dt, smoothed_runoff['ssp585'][basin]['BCC-CSM2-MR'][gmodel][0:100], label = gmodel, color = colors[g])
        axy[c].plot(peak_water_dt['ssp585'][basin]['BCC-CSM2-MR'][gmodel], smoothed_runoff['ssp585'][basin]['BCC-CSM2-MR'][gmodel].loc[peak_water_dt['ssp585'][basin]['BCC-CSM2-MR'][gmodel]], marker = 'o', color = fill_colors[g])

        axy[c].set_xlabel('Year')
        axy[c].set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2000, 2025, 2050, 2075, 2100])

        if c == 0:                                                                    #Setting basin labels
            axy[c].set_ylabel('Smoothed runoff' + r' $[km^3]$')
        
    #Adding in text of which basin
    row_max = max(axy[c].get_ylim())
    row_max_values.append(row_max)
    
    y_coord = row_max_values[b] * 1.028
    axy[c].text(pd.to_datetime('2038-01-01'), y_coord, basin)
    
                      
green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 

axy[0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.15, (0.0167*len(basins))), ncol=3)

plt.suptitle('Smoothed Runoff Projections and Calculated Peak Water Times for all Basins–Rolling mean', x=0.52, y=1.12)

In [ ]:
num_rows = 1
num_cols = 4

figy, axy = plt.subplots(num_rows, num_cols, figsize=(10, 4))
row_max_values = []
r = -1
c = 0

basins_slice = [ 'SANTA CRUZ', 'COPPER', 'ARAL SEA', 'YUKON']

for b, basin in enumerate(basins_slice):
    if c % 3 == 0:
        r += 1  
    c = (c + 1) % num_cols
    for g, gmodel in enumerate(gmodels):
        for m, GCM in enumerate(modelnames):
            axy[c].plot(yrs_dt, smoothed_runoff['ssp585'][basin][GCM][gmodel][0:100], label = gmodel, color = colors[g])
            axy[c].plot(peak_water_dt['ssp585'][basin][GCM][gmodel], smoothed_runoff['ssp585'][basin][GCM][gmodel].loc[peak_water_dt['ssp585'][basin][GCM][gmodel]], marker = 'o', color = fill_colors[g])
        
            axy[c].set_xlabel('Year')
            axy[c].set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2000, 2025, 2050, 2075, 2100])
        
            if c == 0:                                                                    #Setting basin labels
                axy[c].set_ylabel('Smoothed runoff' + r' $[km^3]$')
        
    #Adding in text of which basin
    row_max = max(axy[c].get_ylim())
    row_max_values.append(row_max)
    
    y_coord = row_max_values[b] * 1.028
    axy[c].text(pd.to_datetime('2038-01-01'), y_coord, basin)
    
                      
green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 

axy[0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.15, (0.0167*len(basins))), ncol=3)

plt.suptitle('Smoothed Runoff Projections and Calculated Peak Water Times for all Basins–Rolling mean', x=0.52, y=1.12)

In [ ]:
#Making sure our avergages line up with masked data avgs
avgs = {}

for g, gmodel in enumerate(gmodels):
    years = []
    for m, GCM in enumerate(modelnames):
        years.append(peak_water_dt['ssp245']['SANTA CRUZ'][GCM][gmodel].year)
    
    # Calculate the mean year
    avg_year = statistics.mean(years)
    
    # Store the average year for the current gmodel
    avgs[gmodel] = avg_year

#print(avgs)

In [ ]:
#Based on Holmgren Code
# We want to put the peak water values in a large array for image.
image_data = {}
im_data_masked = {}
for s, SSP in enumerate(scenarios):
    image_data[SSP] = np.zeros((75, 3, 12))
    for i, basin in enumerate(basins):
        for j, gmodel in enumerate(gmodels): 
            values = [peak_water_year[SSP][basin][GCM][gmodel] for GCM in modelnames]
            values.sort()
            #avg_year = statistics.mean(values)
            image_data[SSP][i, j, :len(values)] = values
    
    image_data[SSP] = image_data[SSP].reshape(image_data[SSP].shape[:-2] + (-1,))
    im_data_masked[SSP] = np.ma.masked_where(image_data[SSP] == 0, image_data[SSP])


In [ ]:
#Storing our mean values
mean_dict = {}
for s, SSP in enumerate(scenarios):
    mean_dict[SSP] = {}
    for i, gmodel in enumerate(gmodels):
        mean_dict[SSP][gmodel] = {}
        start_index = i * 12
        end_index = start_index + 12
        for b, basin in enumerate(basins):
            mean_dict[SSP][gmodel][basin] = datetime.datetime(round(np.mean(im_data_masked[SSP][b, start_index:end_index])),1,1)

In [ ]:
# Calculating the inter-GEM ranges
range_dict = {}
for SSP in mean_dict:
    range_dict[SSP] = {}
    for basin in basins:
        range_dict[SSP][basin] = {}
        mean_values = [mean_dict[SSP][gmodel][basin].year for gmodel in mean_dict[SSP]]
        range_dict[SSP][basin]['range'] = max(mean_values) - min(mean_values)

In [ ]:
# Calculating the inter-GCM ranges
# Storing the inter-GCM ranges
inter_gcm_ranges = {}
for s, SSP in enumerate(scenarios):
    inter_gcm_ranges[SSP] = {}
    for basin in basins:
        inter_gcm_ranges[SSP][basin] = {}
        peak_years = []
        for m, GCM in enumerate(modelnames):
            for g, gmodel in enumerate(gmodels):
                peak_years.append(int(peak_water_year[SSP][basin][GCM][gmodel]))

        # Compute the range of peak water years
        peak_years_range = max(peak_years) - min(peak_years)
        inter_gcm_ranges[SSP][basin]['peak_years_range'] = peak_years_range

In [ ]:
# Create tables for each SSP
Range_comp = {}
for SSP in inter_gcm_ranges:
    # Create a DataFrame to store the ranges
    table_data = []
    for basin in inter_gcm_ranges[SSP]:
        # Extract ranges for each GCM and glacier model
        gcm_range = inter_gcm_ranges[SSP][basin]['peak_years_range']
        glacier_model_range = range_dict[SSP][basin]['range']
        
        # Append data to the table
        table_data.append([basin, gcm_range, glacier_model_range])
    
    # Create DataFrame
    Range_comp[SSP] = pd.DataFrame(table_data, columns=['Basin', 'GCM Range', 'Glacier Model Range'])
    

In [ ]:
# Create tables for each SSP
Range_comp = {}
for SSP in inter_gcm_ranges:
    # Create a DataFrame to store the ranges
    table_data = []
    for basin in inter_gcm_ranges[SSP]:
        # Extract ranges for each GCM and glacier model
        gcm_range = inter_gcm_ranges[SSP][basin]['peak_years_range']
        glacier_model_range = range_dict[SSP][basin]['range']
        
        # Replace zeros in glacier_model_range with NaN
        glacier_model_range = np.where(glacier_model_range == 0, np.nan, glacier_model_range)
        
        # Calculate the ratio of GCM range to Glacier Model range
        ratio = gcm_range / glacier_model_range
        
        # Append data to the table
        table_data.append([basin, gcm_range, glacier_model_range, ratio])
    
    # Create DataFrame
    Range_comp[SSP] = pd.DataFrame(table_data, columns=['Basin', 'GCM Range', 'Glacier Model Range', 'Range Ratio'])

In [ ]:
Range_comp['ssp245']['Range Ratio'].mean()

In [ ]:
#Calculate max and min of glacier model PW projections
max_min_gmodel = {}
for SSP in mean_dict:
    max_min_gmodel[SSP] = {}
    for basin in basins:
        mean_values = [mean_dict[SSP][gmodel][basin].year for gmodel in mean_dict[SSP]]
        max_min_gmodel[SSP][basin] = [min(mean_values), max(mean_values)]

#Calculate max and min of GCM PW projections
max_min_GCM = {}
for SSP in scenarios:
    max_min_GCM[SSP] = {}
    for basin in basins:
        peak_years = []
        for GCM in modelnames:
            for gmodel in gmodels:
                peak_years.append(peak_water_year[SSP][basin][GCM][gmodel])

        max_min_GCM[SSP][basin] = [min(peak_years), max(peak_years)]

In [ ]:
# Calculate the inter-GEM ranges
range_dict_gem = {}
for SSP in mean_dict:
    range_dict_gem[SSP] = {}
    for basin in basins:
        mean_values = [mean_dict[SSP][gmodel][basin].year for gmodel in mean_dict[SSP]]
        range_dict_gem[SSP][basin] = max(mean_values) - min(mean_values)

# Calculate the inter-GCM ranges
inter_gcm_ranges = {}
for SSP in scenarios:
    inter_gcm_ranges[SSP] = {}
    for basin in basins:
        peak_years = []
        for GCM in modelnames:
            for gmodel in gmodels:
                peak_years.append(int(peak_water_year[SSP][basin][GCM][gmodel]))

        inter_gcm_ranges[SSP][basin] = max(peak_years) - min(peak_years)

In [ ]:
SSP = 'ssp245'

# Create a new figure and gridspec layout
fig = plt.figure(figsize=(16, 19))
gs = fig.add_gridspec(1, 4, width_ratios=[1, 1, 1, 1], wspace=0.05)

# Loop over each gmodel and corresponding subplot
for i, gmodel in enumerate(gmodels):
    ax = fig.add_subplot(gs[:, i])
    cmap = plt.get_cmap('RdYlBu')

    # The main colormesh
    start_index = i * 12
    end_index = start_index + 12
    im = ax.pcolormesh(im_data_masked[SSP][::-1, start_index:end_index], 
                       cmap=cmap, edgecolors='face', lw=0)

    # Title for each subplot
    ax.set_title(gmodel, fontsize=11)
    ax.set_xticklabels([])

    # Add labels to the pixels. Mean value.
    for b, basin in enumerate(basins):
        ax.text(6, (74 - b) + 0.4, int(mean_dict[SSP][gmodel][basin].year), ha="center", va="center", color="black",
                fontsize=11, transform=ax.transData)

    # Ylabels and ticks for the first subplot
    if i == 0:
        ax.set_yticks(np.arange(image_data[SSP].shape[0])[::-1] + 0.5)
        y_labels = ax.set_yticklabels(basins, fontsize=11)

        # Adjust the position of y-axis labels
        for label in y_labels:
            label.set_ha('right')  # Horizontal alignment to left
            label.set_va('center')  # Vertical alignment

    else:
        ax.set_yticklabels([])

    # Remove ticks and turn off grid
    ax.tick_params(axis='both', which='both', length=0)
    ax.grid(False)

# Plot the inter-GEM and inter-GCM ranges in the new column
ax_range = fig.add_subplot(gs[:, 3])

# Define offset value
offset = 0.15

# Plot inter-GEM ranges
for i, (basin, (min_val, max_val)) in enumerate(max_min_gmodel[SSP].items()):
    ax_range.plot([min_val, max_val], [i + offset, i + offset], color='blue', marker = 'o', markersize=2.5)

# Plot inter-GCM ranges
for i, (basin, (min_val, max_val)) in enumerate(max_min_GCM[SSP].items()):
    ax_range.plot([min_val, max_val], [i - offset, i - offset], color='orange', marker = 'o', markersize=3)

# Customize the plot
ax_range.set_yticks(np.arange(len(basins)))
ax_range.set_yticklabels('')
ax_range.set_xlim(2000, 2100)  # Set x-axis limits
ax_range.set_xticks(np.arange(2000, 2101, 25))  # Set x ticks at intervals of 50 years
ax_range.set_xticklabels(np.arange(2000, 2101, 25), fontsize=10)  # Set x tick labels
ax_range.set_yticks(np.arange(len(basins)))
#ax_range.set_xlabel('Year')
ax_range.invert_yaxis()  # Invert y-axis to match the other subplots
ax_range.set_title('GCM vs Glacier Model Range', fontsize=11)
tick_list = [2025, 2050, 2075]
for tick in tick_list:
    ax_range.axvline(x = tick, color = 'black', ymax = 0.0015)


# Remove ticks and turn off grid
ax_range.tick_params(axis='both', which='both', length=0)

# Set the number of basins
num_basins = len(basins)

# Set y-axis limits
ax_range.set_ylim(num_basins -0.5, -0.6)

# Create a Colorbar object
norm = Normalize(vmin=np.min(im_data_masked[SSP]), vmax=np.max(im_data_masked[SSP]))
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_axes([0.14, 0.07, 0.55, 0.02])  # Adjust position and size of the colorbar
cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar_ax.set_title('Single GCM Year of Peak Water Projections', y= 1.1)

orange_patch = mpatches.Patch(color='orange', label='GCM Range')
GEM_blue_patch = mpatches.Patch(color='blue', label='Glacier Model Range')

ax_range.legend(handles=[orange_patch, GEM_blue_patch], bbox_to_anchor=(0.88, -0.017), ncol=1, fontsize = 11)

# Adjust layout and save plot
#plt.tight_layout()
fig.suptitle(f'Year of Peak Water for 75 major river basins using SSP 2–4.5', fontsize=14, y=0.91)
file_name = f"AvgPeakWater_{SSP}.pdf"  # Name of the file to save
fpath = os.path.expanduser("~/Desktop/Lizz Research/Paper Figs")  # Get the path to your desktop
save_path = os.path.join(fpath, file_name)
plt.savefig(save_path, dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
# Calculating median PW year for ssp 245
median_values = {}
for basin in basins:
    mean_values = [mean_dict['ssp245'][gmodel][basin].year for gmodel in mean_dict['ssp245']]
    median_values[basin] = pd.Series(mean_values).median()
# Convert the dictionary to a DataFrame
median_PW_245 = pd.DataFrame(list(median_values.items()), columns=['Basin', 'Median_PW'])

In [ ]:
# Calculating median PW year for ssp 245
median_PW = {}
for SSP in scenarios:
    median_values = {}
    for basin in basins:
        mean_values = [mean_dict[SSP][gmodel][basin].year for gmodel in mean_dict[SSP]]
        median_values[basin] = pd.Series(mean_values).median()
    # Convert the dictionary to a DataFrame
    median_PW[SSP] = pd.DataFrame(list(median_values.items()), columns=['Basin', 'Median_PW'])

In [ ]:
# Initialize an empty dictionary to store peak water range for each basin
peak_water_range = {}

# Iterate over each basin
for basin in basins:
    median_values_across_ssps = []
    # Collect median peak water values across SSPs for the current basin
    for SSP, df in median_PW.items():
        median_values_across_ssps.append(df.loc[df['Basin'] == basin, 'Median_PW'].values[0])
    # Calculate the range of peak water values for the current basin
    range_pw = max(median_values_across_ssps) - min(median_values_across_ssps)
    peak_water_range[basin] = range_pw

# Convert the dictionary to a DataFrame
peak_water_range_across_ssps = pd.DataFrame(list(peak_water_range.items()), columns=['Basin', 'Peak_Water_Range_Across_SSPs'])

In [ ]:
basin_to_region = {
    'ALSEK': 1, 'COLUMBIA': 2, 'COLVILLE': 1, 'COPPER': 1, 'FRASER': 2,
    'KUSKOKWIM': 1, 'MACKENZIE': 2, 'NASS': 1, 'NUSHAGAK': 1, 'NELSON': 2,
    'SKAGIT': 2, 'SKEENA': 2, 'STIKINE': 1, 'SUSITNA': 1, 'TAKU': 1,
    'YUKON': 1,

    'DANUBE': 11, 'DRAMSELVA': 8, 'GLOMAA': 8, 'JOKULSA A FJOLLUM': 6, 'KALIXALVEN': 8,
    'LAGARFLJOT': 6, 'LULEALVEN': 8, 'OLFUSA': 6, 'PO': 11, 'RHINE': 11,
    'RHONE': 11, 'SVARTA': 6, 'THJORSA': 6, 'TORNEALVEN': 8,

    'ARAL SEA': 14, 'BRAHMAPUTRA': 15, 'CHUY': 13, 'HAR US NUUR': 10, 'GANGES': 15,
    'INDIGIRKA': 10, 'INDUS': 14, 'IRRAWADDY': 15, 'KAMCHATKA': 10, 'KUBAN': 12,
    'LAKE BALKHASH': 13, 'MEKONG': 13, 'OB': 10, 'SALWEEN': 15, 'TALAS': 13, 'TARIM HE': 14,
    'UVS NUUR': 10, 'YANGTZE': 15, 'YELLOW RIVER': 13, 'YSYK-KOL': 13,

    'AISEN': 17, 'AMAZON': 16, 'AZOPARDO': 17, 'BAKER': 17,
    'BIOBIO': 17, 'CHICO': 17, 'CISNES': 17, 'COLORADO': 17,
    'COPIAPO': 17, 'HUASCO': 17, 'MAGDALENA': 16, 'MAJES': 16,
    'NEGRO': 17, 'OCONA': 16, 'PALENA': 17, 'PASCUA': 17,
    'PUELO': 17, 'RAPEL': 17, 'SANTA': 16, 'SANTA CRUZ': 17,
    'SERRANO': 17, 'TITICACA': 16, 'VALDIVIA': 17, 'YELCHO': 17,

    'CLUTHA': 18
}

# Dictionary mapping region codes to colors
region_colors = {
    1: '#1f77b4',  # blue
    2: '#ff7f0e',  # orange
    6: '#2ca02c',  # green
    8: '#d62728',  # red
    10: '#9467bd', # purple
    11: '#8c564b', # brown
    12: '#e377c2', # pink
    13: '#17becf', # cyan
    14: '#aec7e8', # light blue
    15: '#ffbb78', # peach
    16: '#ff9896', # salmon
    17: '#98df8a', # light green
    18: '#c5b0d5'  # lavender
}

In [ ]:
# Extracting median PW values for ssp245
median_pw_ssp245 = median_PW['ssp245']

# Merging the median_PW['ssp245'] DataFrame with df_peak_water_range_across_ssps on 'Basin'
merged_df = pd.merge(median_pw_ssp245, peak_water_range_across_ssps, on='Basin')

In [ ]:
# Dictionary to store unique region-color mappings
unique_region_colors = {}

# Scatter plot with color coding
plt.figure(figsize=(15, 6))

# Iterate over basin and plot scatter points
for basin, row in merged_df.iterrows():
    region = basin_to_region.get(row['Basin'], 'Other')
    color = region_colors.get(region, 'gray')
    plt.scatter(row['Median_PW'], row['Peak_Water_Range_Across_SSPs'], color=color)

    # Add region-color mappings to the unique_region_colors dictionary
    if region not in unique_region_colors:
        unique_region_colors[region] = color

# Plot settings
plt.title('Median Year of Peak Water vs Peak Water Range Across SSPs', fontsize = 14)
plt.xlabel('Median Year of Peak Water (SSP 2-4.5)', fontsize = 12)
plt.ylabel('Year of Peak Water Range Across SSPs', fontsize = 12)

# Sort the unique_region_colors dictionary by region keys
sorted_unique_region_colors = {k: v for k, v in sorted(unique_region_colors.items())}

# Create legend with sorted unique region-color mappings
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label,)
           for label, color in sorted_unique_region_colors.items()]
plt.legend(handles=handles, title='RGI Region', loc='upper left', ncol = 2)
fpath = os.path.expanduser("~/Desktop/Lizz Research/Paper Figs/")  # Get the path to your desktop
file_name = 'PW_vs_SSPrange.pdf'  # Name of the file to save
save_path = os.path.join(fpath, file_name)
plt.savefig(save_path, dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
# # Scatter plot with color coding
# plt.figure(figsize=(10, 8))

# # Iterate over basin and plot scatter points
# for basin, row in merged_df.iterrows():
#     region = basin_to_region.get(row['Basin'], 'Other')
#     color = region_colors.get(region, 'gray')
#     plt.scatter(row['Median_PW'], row['Peak_Water_Range_Across_SSPs'], color=color)

#     # Add region-color mappings to the unique_region_colors dictionary
#     if region not in unique_region_colors:
#         unique_region_colors[region] = color

# # Perform linear regression and compute R-squared value and slope
# x = merged_df['Median_PW']
# y = merged_df['Peak_Water_Range_Across_SSPs']
# slope, intercept, r_value, p_value, std_err = linregress(x, y)

# # Plot linear regression line
# plt.plot(x, slope * x + intercept, color='black', linestyle='--', label=f'Regression Line\n$R^2$: {r_value**2:.2f}, Slope: {slope:.2f}')

# # Plot settings
# plt.title('Median Year of Peak Water vs Peak Water Range Across SSPs')
# plt.xlabel('Median Year of Peak Water (SSP 2-4.5)')
# plt.ylabel('Year of Peak Water Range Across SSPs')

# # Sort the unique_region_colors dictionary by region keys
# sorted_unique_region_colors = {k: v for k, v in sorted(unique_region_colors.items())}

# # Create legend with sorted unique region-color mappings
# handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label,)
#            for label, color in sorted_unique_region_colors.items()]
# # Add legend for regression line
# handles.append(plt.Line2D([0], [0], color='black', linestyle='--', label=f'Regression Line\n$R^2$: {r_value**2:.2f}, Slope: {slope:.2f}'))
# plt.legend(handles=handles, title='RGI Region', loc='upper left', ncol = 2)

# # Save plot
# plt.savefig(save_path, dpi=500, bbox_inches='tight')
# plt.show()

In [ ]:
interannual_variability = {gmodel: {SSP: {basin: {GCM: {} for GCM in modelnames} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
annual_var_median = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}

for gmodel in gmodels:
    for SSP in scenarios:
        for basin in basins:
            for GCM in modelnames:
                runoff_data = smoothed_runoff[SSP][basin][GCM][gmodel]
                variability = np.std(runoff_data)  # can choose other metrics like 25-75%-ile spread if needed
                interannual_variability[gmodel][SSP][basin][GCM] = variability
            annual_var_median[gmodel][SSP][basin] = median_value = statistics.median(interannual_variability[gmodel][SSP][basin].values())

In [ ]:
# # Initialize an empty DataFrame
# mean_df = pd.DataFrame(index=basins, columns=gmodels)

# # Calculate mean values for each combination of basin and gmodel
# for b, basin in enumerate(basins):
#     for g, gmodel in enumerate(gmodels):
#         # Calculate the mean value for the corresponding slice of im_data_masked[SSP]
#         mean_value = im_data_masked[SSP][b, g*12:(g*12)+12].mean()
#         mean_df.loc[basin, gmodel] = mean_value

# #Adding a column providing the std deviation
# mean_df['Std Dev'] = mean_df.std(axis=1)

In [ ]:
# color_map = {'OGGM': 'blue', 'GloGEM': 'green', 'PyGEM': 'purple'}
# SSP = 'ssp245'

# for gmodel in gmodels:
#     for basin in basins:
#         x = annual_var_median[gmodel]['ssp245'][basin]
#         y = mean_df['Std Dev']
#         plt.scatter(x, y, color=color_map[gmodel], label=gmodel)

# plt.xlabel('Inter-annual Variability')
# plt.ylabel('Difference of Peak Year to Median Peak Year')
# plt.title('Peak Year Difference vs. Inter-annual Variability')
# #plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
#Creating visual with just a few basins
selected_basins = ['COPPER', 'INDUS', 'YUKON', 'JOKULSA A FJOLLUM', 'TARIM HE', 'PASCUA'] 
image_data_select = {}
im_data_masked_select = {}
for s, SSP in enumerate(scenarios):
    image_data_select[SSP] = np.zeros((6, 3, 12))
    for i, basin in enumerate(selected_basins):
        for j, gmodel in enumerate(gmodels): 
            values = [peak_water_year[SSP][basin][GCM][gmodel] for GCM in modelnames]
            values.sort()
            #avg_year = statistics.mean(values)
            image_data_select[SSP][i, j, :len(values)] = values
    
    image_data_select[SSP] = image_data_select[SSP].reshape(image_data_select[SSP].shape[:-2] + (-1,))
    im_data_masked_select[SSP] = np.ma.masked_where(image_data_select[SSP] == 0, image_data_select[SSP])

In [ ]:
# SSP = 'ssp245'

# # Main figure
# fig, ax = plt.subplots(figsize=(7.5, 1.5))
# # color axes
# divider = make_axes_locatable(ax)
# #cax = divider.append_axes('bottom', size='2%', pad=0.1)
# # image data, mask bad values
# # Some utility vars.
# xlen = image_data_select[SSP].shape[1]
# ylen = image_data_select[SSP].shape[0]
# # cmap
# cmap = plt.get_cmap('RdYlBu')
# #cmap.set_bad(color='lightgrey')
# # The main colormesh
# im = ax.pcolormesh(im_data_masked_select[SSP][::-1], cmap=cmap,
#                    edgecolors='face', lw=0)
# # Add labels to the pixels. Median value.
# for i in range(ylen):
#     for j in range(3):  # Adjusted to 3 columns for gmodels
#         # Mean value
#         mean = im_data_masked_select[SSP][i, j*12:(j*12)+12].mean()
#         ax.text(7+12*j, (6-i)-0.5, int(mean),
#                 ha="center", va="center", color="black",
#                 fontsize=6, transform=ax.transData,
#                )

# # Ylabels and ticks
# ax.set_yticks(np.arange(image_data_select[SSP].shape[0])[::-1]+0.5)
# y_labels = ax.set_yticklabels(selected_basins)

# # Adjust the position of y-axis labels
# for label in y_labels:
#     label.set_ha('right')  # Horizontal alignment
#     label.set_va('center')  # Vertical alignment
#     label.set_x(label.get_position()[0] - 0.02)  # Adjust the position by adding a small value

# # Xlabels and ticks. Loop cause I'm too lazy to deal with tickers.
# ax.set_xticklabels([])
# for i, gmodel in enumerate(gmodels):  # Loop over gmodels
#     ax.text(7+i*12, 6.5, gmodel, ha='center', va='center',
#            transform=ax.transData,
#            fontsize=8)

# #Remove ticks
# ax.tick_params('x', top=False, bottom = False, pad=-5)
# # Change y tick padding.
# ax.tick_params('y', pad=-4)
# # Labelsize of yticks.
# ax.tick_params('y', labelsize=7)
# #Add the colorbar.
# fig.colorbar(im, cax=cax, orientation='horizontal')
# #Colorbar tick params.
# cax.tick_params('both', labelsize=6, width=0, length=2.5, pad=0.05)
# cax.set_xlabel('Year', fontsize=8)
# #Turn off grid.
# #ax.grid(None)

# # Title
# #ax.set_title(f'Peak water estimations for selected basins using {SSP}', fontsize=11,
#              #x=0.5, y=1.2, ha='center', va="center")

# # Save the plot to your desktop
# desktop_path = os.path.expanduser("~/Desktop/Lizz Research/Paper Figs/")  # Get the path to your desktop
# file_name = f"AvgPeakWater_{SSP}_selected_basins.png"  # Name of the file to save

# # Combine the desktop path and file name to create the full file path
# save_path = os.path.join(desktop_path, file_name)

# # Save the plot as an image
# plt.savefig(save_path, dpi=500, bbox_inches='tight')

# plt.show()

Making all basin runoff figures:

In [ ]:
import statistics

# Define the dictionaries using nested dictionary comprehension
multi_GCM_mean = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
GCM_q1 = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
GCM_q3 = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
percent_change_mean = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
percent_change = {gmodel: {SSP: {basin: {GCM: {} for GCM in modelnames} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
PC_q1 = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
PC_q3 = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}

# percent_change_2000 = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
# relative_change_mean = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
# relative_change_2000 = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}

#Generating data
for gmodel in gmodels:
    for SSP in scenarios:
        for basin in basins:
            dfs1 = [all_rf_data_annual[SSP][basin][GCM][gmodel] for GCM in modelnames]
            multi_GCM_mean[gmodel][SSP][basin] = pd.concat(dfs1, axis=1).median(axis=1)
            #GCM_spread[gmodel][SSP][basin] = (pd.concat(dfs1, axis=1).max(axis=1) - pd.concat(dfs1, axis=1).min(axis=1)).mean()
            
            GCM_q1[gmodel][SSP][basin] = pd.concat(dfs1, axis=1).quantile(q=0.25, axis=1)
            GCM_q3[gmodel][SSP][basin] = pd.concat(dfs1, axis=1).quantile(q=0.75, axis=1)

            percent_change_mean[gmodel][SSP][basin] = ((multi_GCM_mean[gmodel][SSP][basin]- multi_GCM_mean[gmodel][SSP][basin][0:20].mean()) /  multi_GCM_mean[gmodel][SSP][basin][0:20].mean())*100
            for GCM in modelnames:
                percent_change[gmodel][SSP][basin][GCM] = ((all_rf_data_annual[SSP][basin][GCM][gmodel] - all_rf_data_annual[SSP][basin][GCM][gmodel][0:20].mean()) /  all_rf_data_annual[SSP][basin][GCM][gmodel][0:20].mean())*100
            
            dfs2 = [percent_change[gmodel][SSP][basin][GCM] for GCM in modelnames]
            PC_q1[gmodel][SSP][basin] = pd.concat(dfs2, axis=1).quantile(q=0.25, axis=1)
            PC_q3[gmodel][SSP][basin] = pd.concat(dfs2, axis=1).quantile(q=0.75, axis=1)
            
            # percent_change_2000[gmodel][SSP][basin] = ((multi_GCM_mean[gmodel][SSP][basin]- multi_GCM_mean[gmodel][SSP][basin][0]) /  multi_GCM_mean[gmodel][SSP][basin][0]) * 100
            # relative_change_mean[gmodel][SSP][basin] = multi_GCM_mean[gmodel][SSP][basin]- multi_GCM_mean[gmodel][SSP][basin][0:20].mean()
            # relative_change_2000[gmodel][SSP][basin] = multi_GCM_mean[gmodel][SSP][basin]- multi_GCM_mean[gmodel][SSP][basin][0]

In [ ]:
gmodel_spread = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
inter_GCM_spread = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
gmodel_spread_pc = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
inter_GCM_spread_pc = {gmodel: {SSP: {basin: {} for basin in basins} for SSP in scenarios} for gmodel in gmodels}
for gmodel in gmodels:
    for SSP in scenarios:
        for basin in basins:
            dfs1 = [multi_GCM_mean[gmodel][SSP][basin] for gmodel in gmodels]
            df1 = pd.concat(dfs1, axis=1)[20::]
            gmodel_spread[gmodel][SSP][basin] = (df1.max(axis=1) - df1.min(axis=1)).mean()

            dfs2 = [percent_change[gmodel][SSP][basin][GCM] for GCM in modelnames]
            df2 = pd.concat(dfs2, axis=1)[20::]
            inter_GCM_spread_pc[gmodel][SSP][basin] = (df2.max(axis=1) - df2.min(axis=1)).mean()
            
            for GCMs in modelnames:
                dfs = [all_rf_data_annual[SSP][basin][GCM][gmodel] for GCM in modelnames]
                df2 = pd.concat(dfs, axis=1)[20::]
                inter_GCM_spread[gmodel][SSP][basin] = (df2.max(axis=1) - df2.min(axis=1)).mean()

                dfs = [percent_change_mean[gmodel][SSP][basin] for gmodel in gmodels]
                df1 = pd.concat(dfs, axis=1)[20::]
                gmodel_spread_pc[gmodel][SSP][basin] = (df1.max(axis=1) - df1.min(axis=1)).mean()
                

inter_GCM_spread_mean = {SSP: {basin: {} for basin in basins} for SSP in scenarios}
inter_GCM_spread_mean_pc = {SSP: {basin: {} for basin in basins} for SSP in scenarios}
for SSP in scenarios:
    for basin in basins:
        inter_GCM_spread_mean[SSP][basin] = np.mean([inter_GCM_spread[gmodel][SSP][basin] for gmodel in gmodels])
        inter_GCM_spread_mean_pc[SSP][basin] = np.mean([inter_GCM_spread_pc[gmodel][SSP][basin] for gmodel in gmodels])

In [ ]:
dfs = {}

# Populate DataFrames with the calculated values for each SSP
for SSP in scenarios:
    data = {'Glacier_Model_Spread': [gmodel_spread['OGGM'][SSP][basin] for basin in basins],
            'Inter_GCM_Spread': [ inter_GCM_spread_mean[SSP][basin] for basin in basins]}

    # Convert the data dictionary into a DataFrame with basins as index
    dfs[SSP] = pd.DataFrame(data, index=basins)

# Calculate the ratio of inter-GCM spread to inter-glacier-model spread for each SSP
for SSP, df in dfs.items():
    dfs[SSP]['Spread_Ratio'] = dfs[SSP]['Inter_GCM_Spread'] / dfs[SSP]['Glacier_Model_Spread']


In [ ]:
dfs['ssp245']['Spread_Ratio'].nsmallest(20)

In [ ]:
# Check if any values in the 'Ratio_Inter_Spread' column are less than 1
less_than_1 = (dfs['ssp245']['Spread_Ratio'] < 1).sum()

if less_than_1 > 0:
    print(f"There are {less_than_1} values in 'ssp245' DataFrame that are less than 1.")
else:
    print("No values less than 1 found in 'ssp245' DataFrame.")

In [ ]:
# # Select SSP
# SSP = 'ssp245'

# # Initialize a list to store inter-GCM range values for each GEM
# inter_GCM_values_per_GEM = {gmodel: {basin: [] for basin in basins} for gmodel in gmodels}

# # Extract inter-GCM range for each basin for each GEM
# for gmodel in gmodels:
#     for basin in basins:
#         inter_GCM_values_per_GEM[gmodel][basin] = GCM_spread[gmodel][SSP][basin]

# # Calculate inter-GEM range for each basin
# inter_GEM_values = {basin: gmodel_spread[gmodel][SSP][basin] for basin in basins}

# # Create lists for plotting
# basin_names = list(inter_GEM_values.keys())

# # Set the width of the bars
# bar_width = 0.2  # Adjust as needed

# # Set the x positions for the groups
# index = np.arange(len(basin_names))

# # Normalize the values
# max_inter_GEM = max(inter_GEM_values.values())
# normalized_inter_GEM = [val / max_inter_GEM for val in inter_GEM_values.values()]

# # Plotting
# plt.figure(figsize=(15, 6))

# # Plot inter-GEM range
# plt.bar(index, normalized_inter_GEM, bar_width, color='r', alpha=0.5, label='Inter-GEM Range')

# # Plot inter-GCM range for each GEM
# for i, gmodel in enumerate(gmodels):
#     inter_GCM_values = [inter_GCM_values_per_GEM[gmodel][basin] for basin in basin_names]
#     max_inter_GCM = max(inter_GCM_values)
#     normalized_inter_GCM = [val / max_inter_GCM for val in inter_GCM_values]
#     plt.bar(index + (i + 1) * bar_width, normalized_inter_GCM, bar_width, alpha=0.5, label=f'Inter-GCM Range ({gmodel})')

# plt.xlabel('Basin')
# plt.ylabel('Normalized Range')
# plt.title('Normalized Inter-GCM and Inter-GEM Ranges for SSP245')
# plt.xticks(index + bar_width * (len(gmodels) / 2), basin_names, rotation=90)
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
from datetime import datetime
#Examing our smoothed datasets and the assigned peak water times
num_rows = 15
num_cols = 5

SSP = 'ssp245'
SSPnum = SSP[-3::]

figy, axy = plt.subplots(num_rows, num_cols, figsize=(22, 28))
plt.subplots_adjust(hspace=0.2, wspace=0.2)

row_max_values = []
r = -1
c = 0

basin_names = list(basins.keys())

for b, basin in enumerate(basin_names):
    if c % 5 == 0:
        r += 1  
    for g, gmodel in enumerate(gmodels):
        ax = axy[r, c]
        ax.plot(yrs_dt, multi_GCM_mean[gmodel][SSP][basin][0:100], label=gmodel, color=colors[g])
        ax.plot(yrs_dt, GCM_q1[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
        ax.plot(yrs_dt, GCM_q3[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
        ax.fill_between(yrs_dt, GCM_q1[gmodel][SSP][basin][0:100], GCM_q3[gmodel][SSP][basin][0:100], color=fill_colors[g], alpha=0.5)
        ax.axvline(x= mean_dict[SSP][gmodel][basin], color=colors[g], linestyle='--')
        ax.text(1, 0.975, basin, ha='right', va='top', transform=ax.transAxes, fontsize=15)

    if r == 14:
        ax.set_xlabel('Year', fontsize = 15)
        ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                      [2000, 2025, 2050, 2075, 2100], rotation=45, fontsize = 13.5)
    else:
        ax.set_xlabel(None)
        ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                      ['', '', '', '', ''])
    if c == 0:
        ax.set_ylabel(r'$[km^3]$', fontsize = 15)
    c = (c + 1) % num_cols
axy[0,0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.98, 1.55), ncol=3, fontsize = 18)

plt.suptitle(f"Runoff Projections for SSP {SSPnum} ", x=0.52, y=0.915, fontsize = 24)

plt.savefig(f"/Users/finnwimberly/Desktop/Lizz Research/AGU Figures/RFallbasins_{SSP}.pdf", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Creating visual with just a few basins
selected_basins = ['YUKON','GLOMAA', 'RHONE', 'ARAL SEA', 'SERRANO'] 
image_data_select = {}
im_data_masked_select = {}
for s, SSP in enumerate(scenarios):
    image_data_select[SSP] = np.zeros((5, 3, 12))
    for i, basin in enumerate(selected_basins):
        for j, gmodel in enumerate(gmodels): 
            values = [peak_water_year[SSP][basin][GCM][gmodel] for GCM in modelnames]
            values.sort()
            #avg_year = statistics.mean(values)
            image_data_select[SSP][i, j, :len(values)] = values
    
    image_data_select[SSP] = image_data_select[SSP].reshape(image_data_select[SSP].shape[:-2] + (-1,))
    im_data_masked_select[SSP] = np.ma.masked_where(image_data_select[SSP] == 0, image_data_select[SSP])

In [ ]:
gmodel_spread['OGGM']['ssp245']['SERRANO']

In [ ]:
num_rows = 4  # Number of SSPs
num_cols = 5  # Number of basins

basins_select = ['YUKON','GLOMAA', 'RHONE', 'ARAL SEA', 'SERRANO']
basinstext = ['Yukon', 'Glomaa', 'Rhone', 'Aral Sea',  'Serrano']
regions = ['(RGI 1)', '(RGI 8)', '(RGI 11)', '(RGI 14)',  '(RGI 17)']

linestyles = ['dashdot', 'dotted', 'dashed']

scenarios = ['ssp126','ssp245','ssp370','ssp585']
scenarios_text = ['SSP 1-2.6','SSP 2-4.5','SSP 3-7.0','SSP 5-8.5']

figy, axy = plt.subplots(num_rows, num_cols, figsize=(22, 11), sharey='col', sharex=True)
plt.subplots_adjust(hspace=0.1, wspace=0.14)

row_max_values = []
s=0
b=0
for s, SSP in enumerate(scenarios):
    for b, basin in enumerate(basins_select):
        ax = axy[s, b]
        for g, gmodel in enumerate(gmodels):
            ax.plot(yrs_dt, multi_GCM_mean[gmodel][SSP][basin][0:100], label=gmodel, color=colors[g])
            ax.plot(yrs_dt, GCM_q1[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
            ax.plot(yrs_dt, GCM_q3[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
            ax.fill_between(yrs_dt, GCM_q1[gmodel][SSP][basin][0:100], GCM_q3[gmodel][SSP][basin][0:100], color=fill_colors[g], alpha=0.5)
            ax.axvline(x = mean_dict[SSP][gmodel][basin], color=colors[g], linestyle=linestyles[g], ymax = 0.78)



        if s == 3:  # Last row
            ax.set_xlabel('Year', fontsize=15)
            ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                          [2000, 2025, 2050, 2075, 2100], rotation=45, fontsize=13.5)
        else:
            ax.set_xlabel(None)
            ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                          ['', '', '', '', ''])
        if b == 0:  # First column
            ax.set_ylabel(r' $[km^3/yr]$', fontsize=15)
            ax.text(-0.4, 0.5, scenarios_text[s], transform=ax.transAxes, fontsize=15, va='center', ha='center', rotation='horizontal')

        if s == 0:
            ax.text(0.5, 1.3, basinstext[b], transform=ax.transAxes, fontsize=15, va='center', ha='center', rotation='horizontal')
            ax.text(0.5, 1.15, regions[b], transform=ax.transAxes, fontsize=15, va='center', ha='center', rotation='horizontal')

        ax.text(0.01, 0.99, f'Glacer Model Range: {gmodel_spread[gmodel][SSP][basin]:.2f} $km^3/yr$', 
                transform=ax.transAxes,
                verticalalignment='top',
                horizontalalignment='left',
                color='black', fontsize=11)
        
        ax.text(0.01, 0.88, f'GCM Range: {inter_GCM_spread_mean[SSP][basin]:.2f} $km^3/yr$',
                transform=ax.transAxes,
                verticalalignment='top',
                horizontalalignment='left',
                color='black', fontsize=11)

         # Increase y-limit by 10%
        if basin != 'YUKON':
            ymin, ymax = ax.get_ylim()
            ax.set_ylim(ymin, ymax * 1.07)
        
figy.legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(0.5, 0.995), loc='upper center', ncol=3, fontsize=15)
plt.suptitle("Total Annual Runoff Projections", fontsize=20, y = 1.02)

plt.savefig("/Users/finnwimberly/Desktop/Lizz Research/Paper Figs/ResultsOverview.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
#import matplotlib.ticker as ticker

#Examing our smoothed datasets and the assigned peak water times
num_rows = 5
num_cols = 4

basins_select = ['YUKON','GLOMAA', 'RHONE', 'ARAL SEA', 'SERRANO']
basinstext = ['Yukon', 'Glomaa', 'Rhone', 'Aral Sea',  'Serrano']
regions = ['(RGI 1)', '(RGI 8)', '(RGI 11)', '(RGI 14)',  '(RGI 17)']

scenarios = ['ssp245','ssp370','ssp585','ssp126']
#SSP = 'ssp126'

figy, axy = plt.subplots(num_rows, num_cols, figsize=(18, 11), sharey = 'row')
plt.subplots_adjust(hspace=0.15, wspace=0.1)

row_max_values = []
r = -1
c = 0

for b, basin in enumerate(basins_select):
    r += 1
    for s, SSP in enumerate(scenarios):
        c += 1  # Increment column index
        c = c % num_cols  # Reset column index when moving to a new row
        for g, gmodel in enumerate(gmodels):
            ax = axy[r, c]
            ax.plot(yrs_dt, multi_GCM_mean[gmodel][SSP][basin][0:100], label=gmodel, color=colors[g])
            ax.plot(yrs_dt, GCM_q1[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
            ax.plot(yrs_dt, GCM_q3[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
            ax.fill_between(yrs_dt, GCM_q1[gmodel][SSP][basin][0:100], GCM_q3[gmodel][SSP][basin][0:100], color=fill_colors[g], alpha=0.5)

        if r == 4:
            ax.set_xlabel('Year', fontsize = 15)
            ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                          [2000, 2025, 2050, 2075, 2100], rotation=45, fontsize = 13.5)
        else:
            ax.set_xlabel(None)
            ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                          ['', '', '', '', ''])
        if c == 0:
            ax.set_ylabel(r' $[km^3/yr]$', fontsize = 15)
            ax.text(-0.35, 0.6, basinstext[r], transform=ax.transAxes, fontsize=15, va='center', ha='center', rotation='horizontal')
            ax.text(-0.35, 0.4, regions[r], transform=ax.transAxes, fontsize=15, va='center', ha='center', rotation='horizontal')



axy[0,0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.1, 1.66), ncol=3, fontsize = 18)

plt.suptitle(f"Total Annual Runoff Projections", x=0.514, y=1, fontsize = 24)
plt.title('SSP 126                               SSP 245                              SSP 370                                 SSP 585', x=-1.15, y=5.58, fontsize = 18)
#plt.savefig(f"/Users/finnwimberly/Desktop/Lizz Research/Paper Figs/ResultsOverview.pdf", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#import matplotlib.ticker as ticker

#Examing our smoothed datasets and the assigned peak water times
num_rows = 5
num_cols = 4

basins_select = ['RHONE', 'ARAL SEA', 'YUKON', 'SERRANO', 'GLOMAA']
basinstext = ['Rhone', 'Aral Sea', 'Yukon', 'Serrano', 'Glomaa']
scenarios = ['ssp245','ssp370','ssp585','ssp126']
#SSP = 'ssp126'

figy, axy = plt.subplots(num_rows, num_cols, figsize=(18, 11), sharey = 'row')
plt.subplots_adjust(hspace=0.15, wspace=0.1)

row_max_values = []
r = -1
c = 0

for b, basin in enumerate(basins_select):
    r += 1
    for s, SSP in enumerate(scenarios):
        c += 1  # Increment column index
        c = c % num_cols  # Reset column index when moving to a new row
        for g, gmodel in enumerate(gmodels):
            ax = axy[r, c]
            ax.plot(yrs_dt, percent_change_mean[gmodel][SSP][basin][0:100], label=gmodel, color=colors[g])
            ax.plot(yrs_dt, PC_q1[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
            ax.plot(yrs_dt, PC_q3[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
            ax.fill_between(yrs_dt, PC_q1[gmodel][SSP][basin][0:100], PC_q3[gmodel][SSP][basin][0:100], color=fill_colors[g], alpha=0.2)

        if r == 4:
            ax.set_xlabel('Year', fontsize = 15)
            ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                          [2000, 2025, 2050, 2075, 2100], rotation=45, fontsize = 13.5)
        else:
            ax.set_xlabel(None)
            ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                          ['', '', '', '', ''])
        if c == 0:
            ax.set_ylabel(r'% Change', fontsize = 15)

axy[0,0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.1, 1.66), ncol=3, fontsize = 18)

plt.suptitle(f"Percent Change in Annual Runoff Relative to 2000-2020 mean", x=0.514, y=1, fontsize = 24)
plt.title('SSP 126                               SSP 245                              SSP 370                                 SSP 585', x=-1.15, y=5.58, fontsize = 18)
plt.savefig(f"/Users/finnwimberly/Desktop/Lizz Research/Paper Figs/RelativeChange_2000-2020mean.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Creating visual with just a few basins
selected_basins = ['YUKON','SERRANO'] 
image_data_select = {}
im_data_masked_select = {}
for s, SSP in enumerate(scenarios):
    image_data_select[SSP] = np.zeros((2, 3, 12))
    for i, basin in enumerate(selected_basins):
        for j, gmodel in enumerate(gmodels): 
            values = [peak_water_year[SSP][basin][GCM][gmodel] for GCM in modelnames]
            values.sort()
            #avg_year = statistics.mean(values)
            image_data_select[SSP][i, j, :len(values)] = values
    
    image_data_select[SSP] = image_data_select[SSP].reshape(image_data_select[SSP].shape[:-2] + (-1,))
    im_data_masked_select[SSP] = np.ma.masked_where(image_data_select[SSP] == 0, image_data_select[SSP])

In [ ]:
basinstext = ['Yukon', 'Serrano']
SSP = 'ssp245'

#fig, axes = plt.subplots(1, 2, figsize=(15, 6), sharex=True)
figy, axes = plt.subplots(1, 2, figsize=(15, 6), sharex = True)
plt.subplots_adjust(hspace=0.15, wspace=0.1)

for b, basin in enumerate(selected_basins):
    ax = axes[b]
    for g, gmodel in enumerate(gmodels):
        ax.plot(yrs_dt, percent_change_mean[gmodel][scenario][basin][0:100], label=gmodel, color=colors[g])
        ax.plot(yrs_dt, PC_q1[gmodel][scenario][basin][0:100], color=colors[g], linewidth=0.4)
        ax.plot(yrs_dt, PC_q3[gmodel][scenario][basin][0:100], color=colors[g], linewidth=0.4)
        ax.fill_between(yrs_dt, PC_q1[gmodel][scenario][basin][0:100], PC_q3[gmodel][scenario][basin][0:100], color=fill_colors[g], alpha=0.05)
        ax.axvline(x = mean_dict[SSP][gmodel][basin], color=colors[g], linestyle=linestyles[g], ymax = 0.72)
        
    ax.set_xlabel('Year', fontsize=14)
    ax.set_ylabel('Percent Change', fontsize=14)
    ax.set_title(f'{basinstext[b]} Basin', fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=10)

    ax.text(0.01, 0.99, f'Glacer Model Range: {gmodel_spread_pc[gmodel][SSP][basin]:.2f} %', 
                transform=ax.transAxes,
                verticalalignment='top',
                horizontalalignment='left',
                color='black', fontsize=11)
        
    ax.text(0.01, 0.94, f'GCM Range: {inter_GCM_spread_mean_pc[SSP][basin]:.2f} %',
            transform=ax.transAxes,
            verticalalignment='top',
            horizontalalignment='left',
            color='black', fontsize=11)
    
figy.legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(0.5, 0.94), loc='upper center', ncol=3, fontsize=15)
plt.suptitle(f"Percent Change in Annual Runoff Relative to 2000-2019 mean (SSP 2-4.5)", fontsize=20)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(f"/Users/finnwimberly/Desktop/Lizz Research/Paper Figs/PercentChangeOverview.pdf", dpi=450, bbox_inches='tight')
plt.show()

In [ ]:
#Yukon
print(12.03/13.99)
print(12.18/70.88)
#Serrano
print(4.74/1.56)
print(10.86/31.93)


#print(5.819376026272578/1.1629260182876144)

In [ ]:
num_rows = 15
num_cols = 5

SSP = 'ssp245'
SSPnum = SSP[-3::]

figy, axy = plt.subplots(num_rows, num_cols, figsize=(22, 28))
plt.subplots_adjust(hspace=0.2, wspace=0.2)

row_max_values = []
r = -1
c = 0
for b, basin in enumerate(basins):
    if c % 5 == 0:
        r += 1  
    c = (c + 1) % num_cols
    for g, gmodel in enumerate(gmodels):
        ax = axy[r, c]
        ax.plot(yrs_dt, percent_change_mean[gmodel][SSP][basin][0:100], label=gmodel, color=colors[g])
        ax.plot(yrs_dt, PC_q1[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
        ax.plot(yrs_dt, PC_q3[gmodel][SSP][basin][0:100], color=colors[g], linewidth=0.4)
        ax.fill_between(yrs_dt, PC_q1[gmodel][SSP][basin][0:100], PC_q3[gmodel][SSP][basin][0:100], color=fill_colors[g], alpha=0.2)
        # Labeling the basin
        ax.text(1, 1, basin, ha='right', va='top', transform=ax.transAxes, fontsize=15)

    if r == 14:
        ax.set_xlabel('Year', fontsize = 15)
        ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                      [2000, 2025, 2050, 2075, 2100], rotation=45, fontsize = 13.5)
    else:
        ax.set_xlabel(None)
        ax.set_xticks([pd.to_datetime('2000'), pd.to_datetime('2025'), pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')],
                      ['', '', '', '', ''])
    if c == 0:
        ax.set_ylabel(r'% Change', fontsize = 15)

axy[0,0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(3.98, 1.55), ncol=3, fontsize = 18)

plt.suptitle(f"Percent Change in Annual Runoff Relative to 2000-2020 mean for SSP {SSPnum} ", x=0.52, y=0.915, fontsize = 24)

plt.savefig(f"/Users/finnwimberly/Desktop/Lizz Research/AGU Figures/RelativeRFallbasins_{SSP}.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df1 = pd.DataFrame(multi_GCM_mean[gmodel][SSP]['SERRANO'])
df2 = pd.DataFrame(multi_GCM_mean[gmodel][SSP]['YUKON'])

# Calculate the absolute percent change from year to year
percent_change1 = df1.pct_change().abs()
percent_change2 = df2.pct_change().abs()


# Sum the absolute percent change for each column
metric1 = percent_change1.sum()
metric2 = percent_change2.sum()

In [ ]:
smoothed_runoff['ssp245']['SERRANO']['NorESM2-MM']

In [ ]:
#Creating a metric of how flat/steep a projections is 
steepness_metric = {}
for gmodel in gmodels:
    steepness_metric[gmodel] = {}
    for SSP in scenarios:
        steepness_metric[gmodel][SSP] = {}
        for basin in basins:
            df = pd.DataFrame(multi_GCM_mean[gmodel][SSP][basin])
            # Calculate the absolute percent change from year to year
            percent_change = df.pct_change().abs()
            # Sum the absolute percent change for each column
            steepness_metric[gmodel][SSP][basin] = percent_change.sum()

mean_steepness_metric = {}
for SSP in scenarios:
    mean_steepness_metric[SSP] = {}
    for basin in basins:
        # Calculate the mean across all glacier models for the given SSP scenario and basin
        mean_steepness_metric[SSP][basin] = np.mean([steepness_metric[gmodel][SSP][basin] 
                                                      for gmodel in gmodels])

In [ ]:
# Extract data for SSP = 'ssp245' from Range_comp DataFrame
ssp245_data = Range_comp['ssp245']

# Plotting the scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(mean_steepness_metric['ssp245'].values(), ssp245_data['Glacier Model Range'].values, color='blue')

# Add labels and title
plt.xlabel('Mean Steepness Metric for SSP ssp245')
plt.ylabel('Glacier Model Range')
plt.title('Scatter Plot of Glacier Model Range vs. Mean Steepness Metric for SSP ssp245')

# Show plot
plt.grid(True)
plt.show()

In [ ]:
out_df = {}
for s, SSP in enumerate(scenarios):
    out_df[SSP] = {}
    for m, model in enumerate(modelnames):
        basin_data_list = []  # Create a list to store data for all basins
        for b, basin in enumerate(basins):
            glo_values = peak_water_year[SSP][basin][model]['GloGEM']
            pygem_values = peak_water_year[SSP][basin][model]['PyGEM']
            oggm_values = peak_water_year[SSP][basin][model]['OGGM']

            # Create a dictionary with values for the current basin
            data = {
                'GloGEM': glo_values,
                'OGGM': oggm_values,
                'PyGEM': pygem_values,
            }
            
            basin_data_list.append(data)  # Append data for the current basin
        
        # Create a DataFrame from the list of basin data
        out_df[SSP][model] = pd.DataFrame(basin_data_list, index=basins)  # Set the index as 'Basin'

In [ ]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Peak Water/'

for SSP in out_df:
    for GCM in out_df[SSP]:
        fname = f"PeakWaterYr{GCM}_{SSP}.csv"

        # Define the full path of the output file
        output_path = os.path.join(output_dir, fname)

        # Save the DataFrame as CSV
        out_df[SSP][GCM].to_csv(output_path, header=True, index=True)